### 1. Load libraries

In [113]:
# import packages
import pandas as pd
import datetime as dt
import glob


### 2. Load data

In [114]:
# Download data from the period 2010-2020 
all_files = glob.glob("./data/datos*.csv")

dfs = []

for filename in all_files:
    df = pd.read_csv(filename, header=0, sep=';')
    dfs.append(df)

df = pd.concat(dfs, axis=0, ignore_index=True)

In [115]:
df.columns = map(str.lower, df.columns)

In [116]:
df.head()

,provincia,municipio,estacion,magnitud,punto_muestreo,ano,mes,d01,v01,d02,...,d27,v27,d28,v28,d29,v29,d30,v30,d31,v31
0,28,79,4,1,28079004_1_38,2017,1,6.0,V,8.0,...,5.0,V,4.0,V,4.0,V,5.0,V,7.0,V
1,28,79,4,1,28079004_1_38,2017,2,8.0,V,4.0,...,4.0,V,4.0,V,0.0,N,0.0,N,0.0,N
2,28,79,4,1,28079004_1_38,2017,3,5.0,V,7.0,...,4.0,V,5.0,V,7.0,V,7.0,V,5.0,V
3,28,79,4,1,28079004_1_38,2017,4,4.0,V,4.0,...,3.0,V,4.0,V,4.0,V,4.0,V,0.0,N
4,28,79,4,1,28079004_1_38,2017,5,4.0,V,5.0,...,5.0,V,5.0,V,4.0,V,4.0,V,5.0,V


### 3. Data cleansing

In [117]:
# Select columns with daily values

days= df.filter(regex=("^d.*$"))

days.head()

,d01,d02,d03,d04,d05,d06,d07,d08,d09,d10,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,6.0,8.0,12.0,9.0,7.0,10.0,10.0,11.0,10.0,12.0,...,7.0,9.0,12.0,9.0,9.0,5.0,4.0,4.0,5.0,7.0
1,8.0,4.0,3.0,3.0,3.0,4.0,5.0,5.0,6.0,8.0,...,12.0,11.0,6.0,6.0,5.0,4.0,4.0,0.0,0.0,0.0
2,5.0,7.0,5.0,3.0,3.0,4.0,6.0,9.0,10.0,10.0,...,4.0,3.0,4.0,4.0,4.0,4.0,5.0,7.0,7.0,5.0
3,4.0,4.0,5.0,7.0,4.0,4.0,5.0,5.0,5.0,6.0,...,5.0,5.0,6.0,4.0,4.0,3.0,4.0,4.0,4.0,0.0
4,4.0,5.0,5.0,6.0,5.0,4.0,4.0,4.0,5.0,4.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0


In [118]:
# Select columns with data validity
validity = df.filter(regex=("^v.*$"))

validity.head()

,v01,v02,v03,v04,v05,v06,v07,v08,v09,v10,...,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31
0,V,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,V
1,V,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,N,N,N
2,V,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,V
3,V,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,N
4,V,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,V


In [119]:
# There are 19219 invalid measurements and 558621 valid
validity.stack().value_counts()

V    563054
N     19529
dtype: int64

In [120]:
# Convert V in null values
validity = validity.replace('V', np.nan)

In [121]:
# Change column names in validty and use 'days' column names
validity.columns = days.columns

In [122]:
# Use combine_first to get valid data
valid_data= validity.combine_first(days)

In [123]:
# Replace 'N' values for np.nan
valid_data= valid_data.replace('N', np.nan)

valid_data.head()

,d01,d02,d03,d04,d05,d06,d07,d08,d09,d10,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,6.0,8.0,12.0,9.0,7.0,10.0,10.0,11.0,10.0,12.0,...,7.0,9.0,12.0,9.0,9.0,5.0,4.0,4.0,5.0,7.0
1,8.0,4.0,3.0,3.0,3.0,4.0,5.0,5.0,6.0,8.0,...,12.0,11.0,6.0,6.0,5.0,4.0,4.0,NaN,NaN,NaN
2,5.0,7.0,5.0,3.0,3.0,4.0,6.0,9.0,10.0,10.0,...,4.0,3.0,4.0,4.0,4.0,4.0,5.0,7.0,7.0,5.0
3,4.0,4.0,5.0,7.0,4.0,4.0,5.0,5.0,5.0,6.0,...,5.0,5.0,6.0,4.0,4.0,3.0,4.0,4.0,4.0,NaN
4,4.0,5.0,5.0,6.0,5.0,4.0,4.0,4.0,5.0,4.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0


In [124]:
cols = valid_data.columns

In [125]:
# Create a new dataframe concatenating columns from df_match_no2 and valid measurements
df_valid = pd.concat([df[['estacion', 'magnitud', 'ano','mes']], valid_data], axis=1)

df_valid.head()

,estacion,magnitud,ano,mes,d01,d02,d03,d04,d05,d06,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,4,1,2017,1,6.0,8.0,12.0,9.0,7.0,10.0,...,7.0,9.0,12.0,9.0,9.0,5.0,4.0,4.0,5.0,7.0
1,4,1,2017,2,8.0,4.0,3.0,3.0,3.0,4.0,...,12.0,11.0,6.0,6.0,5.0,4.0,4.0,NaN,NaN,NaN
2,4,1,2017,3,5.0,7.0,5.0,3.0,3.0,4.0,...,4.0,3.0,4.0,4.0,4.0,4.0,5.0,7.0,7.0,5.0
3,4,1,2017,4,4.0,4.0,5.0,7.0,4.0,4.0,...,5.0,5.0,6.0,4.0,4.0,3.0,4.0,4.0,4.0,NaN
4,4,1,2017,5,4.0,5.0,5.0,6.0,5.0,4.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0


In [126]:
# Use melt to create a column 'day'
df_valid = pd.melt(df_valid,id_vars= ['estacion','magnitud','ano','mes'], value_vars =cols,
                             var_name ='dia', value_name ='medicion')
                                                           
df_valid.head()

,estacion,magnitud,ano,mes,dia,medicion
0,4,1,2017,1,d01,6.0
1,4,1,2017,2,d01,8.0
2,4,1,2017,3,d01,5.0
3,4,1,2017,4,d01,4.0
4,4,1,2017,5,d01,4.0


In [127]:
# Drop rows with NULL values
df_valid = df_valid.dropna()

In [128]:
# Remove 'd' from column 'day'
df_valid['dia'] = df_valid['dia'].str.replace('d', '')

In [129]:
# Create a new column 'date' joining year, month and day
df_valid['fecha'] = (df_valid['ano'].map(str) + "-" + df_valid['mes'].map(str) + "-" 
                                                             + df_valid['dia'].map(str))

In [130]:
# Convert 'date' to datetime
df_valid['fecha'] = pd.to_datetime(df_valid['fecha'], format='%Y-%m-%d')

df_valid.head()

,estacion,magnitud,ano,mes,dia,medicion,fecha
0,4,1,2017,1,01,6.0,2017-01-01
1,4,1,2017,2,01,8.0,2017-02-01
2,4,1,2017,3,01,5.0,2017-03-01
3,4,1,2017,4,01,4.0,2017-04-01
4,4,1,2017,5,01,4.0,2017-05-01


In [131]:
# Drop columns 'year','month','day'
df_valid.drop(columns=['ano', 'mes', 'dia'], inplace=True)

df_valid.head()

,estacion,magnitud,medicion,fecha
0,4,1,6.0,2017-01-01
1,4,1,8.0,2017-02-01
2,4,1,5.0,2017-03-01
3,4,1,4.0,2017-04-01
4,4,1,4.0,2017-05-01


In [23]:
df_valid_no_background = df_valid[(df_valid.station != 58) & (df_valid.station !=24)]

In [132]:
# Use grouby to calculate daily average emission by date and particle.
df_avg_emissions = pd.DataFrame(df_valid_no_background.groupby(['fecha', 'magnitud'])
                                ['medicion']
                                .mean().reset_index())


In [133]:
# Use pivot to turn every particle in a column
df_avg_emissions = pd.pivot(df_avg_emissions, index= 'fecha', columns='magnitud',values='medicion')

In [134]:
# Create list with name of particles and rename columns
particles = ['so2', 'co','no','no2','pm2_5','pm10','nox','o3','tol','ben','ebe','tch','ch4','nmhc']

df_avg_emissions.columns = particles


In [135]:
# Convetir index to datetime

df_avg_emissions.index = pd.to_datetime(df_avg_emissions.index)

In [136]:
df_avg_emissions

,so2,co,no,no2,pm2_5,pm10,nox,o3,tol,ben,ebe,tch,ch4,nmhc
fecha,,,,,,,,,,,,,,
2010-01-01,7.900000,0.322222,7.105263,20.315789,7.000000,10.250000,31.210526,60.583333,1.750000,0.575000,1.000000,1.343333,1.133333,0.210000
2010-01-02,9.500000,0.433333,20.684211,42.368421,10.000000,15.250000,74.000000,27.583333,3.675000,1.000000,1.100000,1.416667,1.213333,0.206667
2010-01-03,11.600000,0.555556,37.421053,54.052632,12.250000,16.625000,111.578947,12.583333,5.960000,1.440000,1.580000,1.543333,1.366667,0.180000
2010-01-04,11.600000,0.511111,31.736842,55.526316,9.000000,9.750000,104.052632,13.416667,5.100000,1.280000,1.860000,1.483333,1.306667,0.180000
2010-01-05,11.900000,0.422222,22.736842,48.368421,8.000000,12.125000,83.263158,23.166667,3.675000,0.725000,1.600000,1.436667,1.256667,0.183333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-26,4.888889,0.200000,2.130435,6.086957,8.000000,13.615385,9.565217,63.785714,0.350000,0.200000,0.183333,1.300000,1.200000,0.096667
2020-04-27,4.777778,0.200000,2.695652,9.130435,5.714286,9.230769,13.086957,75.500000,0.266667,0.133333,0.133333,1.263333,1.153333,0.106667
2020-04-28,4.888889,0.188889,3.043478,10.304348,4.714286,6.846154,14.913043,66.214286,0.316667,0.150000,0.150000,1.276667,1.163333,0.113333
